In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import yfinance as yf
import random
import itertools


In [2]:
df = pd.read_csv('sp_500.csv')
df

,Security,Symbol,GICS_Sector,GICS_Sub_Industry
0,3M,MMM,Industrials,Industrial Conglomerates
1,A. O. Smith,AOS,Industrials,Building Products
2,Abbott,ABT,Health Care,Health Care Equipment
3,AbbVie,ABBV,Health Care,Pharmaceuticals
4,Accenture,ACN,Information Technology,IT Consulting & Other Services
...,...,...,...,...
498,Yum! Brands,YUM,Consumer Discretionary,Restaurants
499,Zebra Technologies,ZBRA,Information Technology,Electronic Equipment & Instruments
500,Zimmer Biomet,ZBH,Health Care,Health Care Equipment
501,Zions Bancorporation,ZION,Financials,Regional Banks


In [3]:
start_date = dt.datetime(2022, 1, 1)
end_date = dt.datetime(2025, 2, 24)

names = df['Symbol'].tolist()[:100]
names = [ticker for ticker in names if ticker not in ['BRK.B', 'CTLT', 'BF.B', 'ABC', 'ATVI']]

n_tickers_selected = 4
n_combinations = 100


all_combinations = list(itertools.combinations(names, n_tickers_selected))


sampled_combinations= random.sample(list(itertools.combinations(names, n_tickers_selected)),n_combinations) 
sampled_combinations


[('MMM', 'AFL', 'A', 'ACGL'),
 ('MO', 'BA', 'CHRW', 'COF'),
 ('ADM', 'ADI', 'BALL', 'BSX'),
 ('ADI', 'BWA', 'BXP', 'CARR'),
 ('ABT', 'AMGN', 'BIO', 'BA'),
 ('AAP', 'AVY', 'BDX', 'COF'),
 ('ALL', 'BKR', 'BRO', 'CDW'),
 ('APA', 'AVY', 'BAC', 'BBWI'),
 ('APA', 'ATO', 'CDNS', 'CAT'),
 ('ALB', 'APA', 'ATO', 'CBRE'),
 ('LNT', 'GOOGL', 'AMD', 'AMT'),
 ('ACN', 'AES', 'APH', 'CZR'),
 ('AMD', 'TECH', 'BR', 'BRO'),
 ('AAP', 'ALK', 'AIG', 'AMAT'),
 ('ABT', 'APA', 'AVGO', 'CPB'),
 ('ACGL', 'ADSK', 'AVB', 'CNP'),
 ('ADBE', 'AKAM', 'CZR', 'CDW'),
 ('AZO', 'WRB', 'CZR', 'CNP'),
 ('AAL', 'AWK', 'BWA', 'CAT'),
 ('AEP', 'BXP', 'CBRE', 'CE'),
 ('ADBE', 'AMGN', 'BALL', 'COF'),
 ('ADM', 'AVY', 'BAC', 'BR'),
 ('ACN', 'GOOGL', 'BAX', 'CNP'),
 ('MMM', 'AEP', 'ADI', 'BSX'),
 ('ABBV', 'AMD', 'BBY', 'BIO'),
 ('AEP', 'APH', 'ANSS', 'KMX'),
 ('ADM', 'ACGL', 'CCL', 'CBOE'),
 ('MMM', 'AKAM', 'AMT', 'CPT'),
 ('ABT', 'AFL', 'ALLE', 'CBRE'),
 ('AFL', 'ALLE', 'MO', 'CAH'),
 ('A', 'BWA', 'CZR', 'CBOE'),
 ('ALGN', 'BALL', 

In [4]:
all_portfolios = []
best_portfolios = [] 

risk_free_rate = 0.0415


for combination in sampled_combinations:
    data = yf.download(list(combination), start=start_date, end=end_date, progress=False)["Close"]
    returns = data.pct_change().dropna()
    covariance_matrix = returns.cov()

    annual_returns = returns.mean() * 252
    annual_cov = covariance_matrix * 252

    num_portafolios = 1000

    weights_array = np.zeros((num_portafolios, n_tickers_selected))
    returns_array = np.zeros(num_portafolios)
    risk_array = np.zeros(num_portafolios)
    sharpe_array = np.zeros(num_portafolios)

    for i in range(num_portafolios):
        weights = np.random.random(n_tickers_selected)
        weights = weights / np.sum(weights)

        weights_array[i] = weights

        portfolio_return = np.dot(annual_returns, weights)
        portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(annual_cov, weights)))
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_risk

        returns_array[i] = portfolio_return
        risk_array[i] = portfolio_risk
        sharpe_array[i] = sharpe_ratio

    portfolios = pd.DataFrame({
        "Returns": returns_array,
        "Risk": risk_array,
        "Sharpe Ratio": sharpe_array
    })

    for j, symbol in enumerate(combination):
        portfolios[symbol + " Weight"] = weights_array[:, j]
        

    all_portfolios.append(portfolios)

    best_portfolio = portfolios.nlargest(1, 'Sharpe Ratio')
    best_portfolios.append(best_portfolio)

best_portfolios_df = pd.concat(best_portfolios).reset_index(drop=True)

simulaciones = 10000
VaR_array = []

for index, row in best_portfolios_df.iterrows():
    mu_portafolio = row["Returns"] * (14/252)
    sigma_portafolio = row["Risk"] * np.sqrt(14/252)
    
    
    
    simulated_returns = np.random.normal(loc=mu_portafolio, scale=sigma_portafolio, size=simulaciones)
    
   
    VaR = -np.percentile(simulated_returns, 5) 
    VaR_array.append(VaR)

best_portfolios_df["VaR"] = VaR_array

YF.download() has changed argument auto_adjust default to True


In [5]:
best_portfolios_df2 = best_portfolios_df.fillna(0)
best_portfolios_10 = best_portfolios_df2.nlargest(10, 'Sharpe Ratio')
best_portfolios_10

,Returns,Risk,Sharpe Ratio,MMM Weight,AFL Weight,A Weight,ACGL Weight,MO Weight,BA Weight,CHRW Weight,...,T Weight,AMP Weight,APTV Weight,AJG Weight,GOOG Weight,AAPL Weight,AXP Weight,ADP Weight,BLK Weight,VaR
29,0.295550,0.192461,1.320010,0.000000,0.290392,0.0,0.000000,0.672344,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058662
91,0.280272,0.188049,1.269737,0.000000,0.000000,0.0,0.013745,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.056906
57,0.295586,0.200463,1.267497,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.040036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061933
60,0.292058,0.207664,1.206554,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065430
36,0.276757,0.196696,1.196044,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.081904,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061839
66,0.264273,0.189407,1.176158,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.729332,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057200
23,0.268196,0.195448,1.159878,0.130356,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061159
14,0.408153,0.356141,1.029517,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.112404
2,0.240469,0.193373,1.028939,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061608
68,0.245821,0.200249,1.020331,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065945
